In [1]:
import numpy as np
import six
import sys
import os
import traceback
import re
import pickle
from copy import deepcopy

from chainer import cuda
from context2vec.common.context_models import Toks
from context2vec.common.model_reader import ModelReader
import sklearn
import pandas as pd
import logging
from scipy.stats import spearmanr
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim
import math




[nltk_data] Downloading package stopwords to /home/ql261/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def produce_top_n_simwords(w_filter,context_embed,n_result,index2word):
        #assume that w_filter is already normalized
        context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
        similarity_scores=[]
        print('producing top {0} simwords'.format(n_result))
        similarity = (w_filter.dot(context_embed)+1.0)/2
        top_words_i=[]
        top_words=[]
        count = 0
        for i in (-similarity).argsort():
                    if xp.isnan(similarity[i]):
                        continue
                    print('{0}: {1}'.format(str(index2word[i]), str(similarity[i])))
                    count += 1
                    top_words_i.append(i)
                    top_words.append(index2word[i])
                    similarity_scores.append(similarity[i])
                    if count == n_result:
                        break

        top_vec=w_filter[top_words_i,:]
        
        return top_vec,np.array(similarity_scores),top_words
    
def top_mutual_sim(top_vec,similarity_scores):

    #normalize the top_vec
    s = np.sqrt((top_vec * top_vec).sum(1))
    s[s==0.] = 1.
    top_vec /= s.reshape((s.shape[0], 1))
    
    # substitutes' similarity to sentence (similarity_scores) as weight matrix to mutual similarity
    max_score=similarity_scores[0]
    similarity_scores=np.array(similarity_scores)
    sim_weights=(similarity_scores+similarity_scores.reshape(len(similarity_scores),1))/2.0
    #weighted by the maximum score in the substitutes (highre max score means the context is more certain about the substitutes)
    sim_weights=(sim_weights/float(sum(sum(sim_weights))))*max_score
    # dot product weighted by substitute probability (sim_weights)
    inf_score=sum(sum(top_vec.dot(top_vec.T)*sim_weights))
    return inf_score

def top_cluster_density(top_vec,similarity_scores):
    #normalize the top_vec
    s = np.sqrt((top_vec * top_vec).sum(1))
    s[s==0.] = 1.
    top_vec = top_vec/ s.reshape((s.shape[0], 1))
    
    #perform the centroid
    max_score=similarity_scores[0]
    similarity_scores=np.array(similarity_scores).reshape(len(similarity_scores),1)/sum(similarity_scores)
    centroid_vector=sum(top_vec*similarity_scores)
    # average of cosine distance to the centroid,weighted by max scores
    inf_score=sum(top_vec.dot(centroid_vector))/len(top_vec)*max_score
    return inf_score

In [3]:
top_cluster_density(np.array([[1.,1.,1.],[4.,4.,4.]]),np.array([0.3,0.1]))
# np.linalg.norm([0.57735027,0.57735027, 0.57735027])

0.3

In [27]:
def load_w2salience(w2salience_f,weight_type):
    w2salience={}
    with open(w2salience_f) as f:
        for line in f:
            if line.strip()=='':
                continue
            w,w_count,s_count=line.strip().split('\t')
            if weight_type==INVERSE_W_FREQ:
                w2salience[w]=1/float(w_count)
            elif weight_type==INVERSE_S_FREQ:
                w2salience[w]=math.log(1+84755431/float(s_count))
    return w2salience

def skipgram_context(model,words,pos,weight=None,w2entropy=None):
    context_wvs=[]
    weights=[]
    for i,word in enumerate(words):
        if i != pos: #surroudn context words
            try:
                if weight ==LDA:
                    if word in w2entropy and word in model:
                        print (word,w2entropy[word])
                        weights.append(1/(w2entropy[word]+1.0))
                        context_wvs.append(model[word])
                elif weight in [INVERSE_W_FREQ,INVERSE_S_FREQ]:
                    if word in w2entropy and word in model:
                        print (word,w2entropy[word])
                        weights.append(w2entropy[word])
                        context_wvs.append(model[word])
                else:
                    #equal weights per word
                    context_wvs.append(model[word])
                    weights.append(1.0)
            except KeyError as e:
                print ('==warning==: key error in context {0}'.format(e))
    context_embed=sum(np.array(context_wvs)*np.array(weights).reshape(len(weights),1))#/sum(weights)
    return sum(weights),context_embed #  will be normalized later

def lg_model_out_w2v(top_words,w_target,word2index_target):
        # lg model substitutes in skipgram embedding
        top_vec=[]
        index_list=[]
        for i,word in enumerate(top_words):
            try :
                top_vec.append(w_target[word2index_target[word]])
                index_list.append(i)
            except KeyError as e:
                print (e)
        return np.array(top_vec),index_list
    
def context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
    #produce context representation and infromative score for each context
    test_s=test_s.replace(test_w, ' '+test_w+' ')
    print(test_s)
    words=test_s.split()
    pos=words.index(test_w)
    
    score=1.0 #default score
    
    # Decide on the model
    if model_type=='context2vec':
        context_embed= model.context2vec(words, pos)
        context_embed_out=context_embed
    
    elif model_type=='skipgram':
        score,context_embed=skipgram_context(model,words,pos,weight,w2entropy)
        context_embed_out=context_embed
        
    elif model_type=='context2vec-skipgram':
        # context2vec substitutes in skipgram space
        context_embed= model.context2vec(words, pos)
        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        top_vec,index_list=lg_model_out_w2v(top_words,w_target,word2index_target) 
        sim_scores=sim_scores[index_list] #weighted by substitute probability
        context_embed_out=sum(top_vec*((sim_scores/sum(sim_scores)).reshape(len(sim_scores),1)))
    else:
        print ('model type {0} not recognized'.format(model_type))
        sys.exit(1)
        
        
    #decide on weight per sentence
    if weight==TOP_MUTUAL_SIM:
        print (weight)
#         if word2index_target==None: #not context2vec-skipgram
#             context2vec word embedding space neighbours
        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        #skipgram word embedding space neighbours when context2vec-skipgram
        score=top_mutual_sim(top_vec,sim_scores)
        print (score)
    elif weight==TOP_CLUSTER_DENSITY:
        print (weight)
#         if word2index_target==None: #not context2vec-skipgram
#             context2vec word embedding space neighbours
        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        score=top_cluster_density(top_vec,sim_scores)
        print (score)
        
    elif weight=='learned':
        print ('learned not implemented')
    elif weight=='gaussian':
        print ('gaussian not implemented')
    elif weight ==False or weight in [LDA,INVERSE_S_FREQ,INVERSE_W_FREQ]:
        score=score
    else:
        print ('weight mode {0} not recognized'.format(weight))
    return score,context_embed_out

def additive_model(f_w,test_ss,test_w, model_type,model,n_result,w_filter,index2word,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
    #produce context representation across contexts using weighted average
    
    context_out=[]
    context_weights=[]
    for test_s in test_ss.split('@@'):
        test_s=test_s.strip()
        #produce context representation with scores
        score,context_embed=context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight,w2entropy,w_target,word2index_target,index2word_target)
        print ('weight is {0}'.format(score))
        context_out.append(context_embed)
        context_weights.append(score)
    
    
    #sum representation across contexts
    context_out=np.array(context_out)
    norm_weights=np.array(context_weights).reshape(len(context_weights),1)/float(sum(context_weights))
    f_w.write(','.join([str(i[0]) for i in norm_weights])+'\n')
    print ('normalized weight: \n  {0}'.format(norm_weights))
    
    if model_type=='skipgram':
        # context representation by weighted sum of all context words in all contexts
        context_avg=sum(context_out)/sum(context_weights)
    else:
        # context represenatation by weighted sum of contexts
        context_avg=sum(norm_weights*context_out)
    
    
    # check new embedding neighbours

    print('producing top {0} words for new embedding'.format(n_result))
    if index2word_target==None:
        top_vec,scores,top_words=produce_top_n_simwords(w,context_avg,n_result,index2word)
    else:
        #print the target space neighbours for context2vec-skipgram
        print (w_target.shape)
        top_vec,scores,top_words=produce_top_n_simwords(w_target,context_avg,n_result,index2word_target)
    
    return context_avg




In [5]:
def filter_w(w,word2index,index2word):
    #filter out words with no letters in, and stopwords
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    index2word_filter={}
    word2index_filter={}
    index_filter2index=[]
    counter=0
    for word in word2index:
            if word not in stopw:
                    index_filter2index.append(word2index[word])
                    word2index_filter[word]=counter
                    index2word_filter[counter]=word
                    counter+=1
    w_filter= w[index_filter2index,:]
    return w_filter,word2index_filter,index2word_filter

def rm_stopw_context(model):
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    
    model={word:model.wv.__getitem__(word) for word in model.wv.vocab if word not in stopw}
    return model




In [6]:
def eval_chimera(chimeras_data_f,context_model,model_type,n_result,w,index2word,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
    chimeras_data_dir='/'.join(chimeras_data_f.split('/')[:-1])
    num_sent=chimeras_data_f.split('/')[-1].split('.')[1][1]
    print (chimeras_data_dir)
    print (num_sent)
    with open(chimeras_data_dir+'/weights_{0}_{1}_{2}'.format(num_sent,model_type,str(weight)),'w') as f_w:
        spearmans=[]
        data=pd.read_csv(os.path.join(chimeras_data_f),delimiter='\t',header=None)

        for index, row in data.iterrows():
            golds=[]
            model_predict=[]
            probes=[]
            #compute context representation
            if weight!='learned':
                context_avg=additive_model(f_w,row[1].lower(),'___', model_type,context_model,n_result,w,index2word,weight,w2entropy,w_target,word2index_target,index2word_target)
            context_avg = context_avg / xp.sqrt((context_avg * context_avg).sum())

            #cosine similarity with probe embedding
            for gold,probe in zip(row[3].split(','),row[2].split(',')):
                try:
                    if index2word_target==None:
                        probe_w_vec=xp.array(w[word2index[probe]])
                    else:
                        probe_w_vec=xp.array(w_target[word2index_target[probe]])
                    probe_w_vec=probe_w_vec/xp.sqrt((probe_w_vec*probe_w_vec).sum())
                    cos=probe_w_vec.dot(context_avg)
                    if xp.isnan(cos):
                        continue
                    else:
                        model_predict.append(cos)
                        golds.append(gold)
                        probes.append(probe)
                except KeyError as e:
                    print ("====warning key error for probe=====: {0}".format(e))
            print ('probes',probes)
            print ('gold',golds)
            print ('model_predict',model_predict)
            sp=spearmanr(golds,model_predict)[0]
            print ('spearman correlation is {0}'.format(sp))
            if not math.isnan(sp):
                spearmans.append(sp)
        print ("AVERAGE RHO:",float(sum(spearmans))/float(len(spearmans)))

In [14]:
TOP_MUTUAL_SIM='top_mutual_sim'
TOP_CLUSTER_DENSITY='top_cluster_density'
LDA='lda'
INVERSE_S_FREQ='inverse_s_freq'
INVERSE_W_FREQ='inverse_w_q'
WEIGHT_DICT={0:False,1:TOP_MUTUAL_SIM,2:LDA,3:INVERSE_S_FREQ,4:INVERSE_W_FREQ,5:TOP_CLUSTER_DENSITY}


if __name__=="__main__":
    
    #params read in
    if sys.argv[0]=='/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py':
        
        data='./eval_data/data-chimeras/dataset.l4.fixed.test.txt.punct'

        weight=WEIGHT_DICT[5]
        
#         ##context2vec
##         model_param_file='../models/context2vec/model_dir/context2vec.ukwac.model.params'
#         model_param_file='../models/context2vec/model_dir/MODEL-wiki.params.14'
        
#         model_type='context2vec'
#         context_rm_stopw=0

# ####skipgram
#         model_param_file='../models/wiki_all.model/wiki_all.sent.split.model'
#         model_type='skipgram'
#         context_rm_stopw=1
#         weight='inverse_w_freq'
#         w2salience_f='../corpora/corpora/wiki.all.utf8.sent.split.tokenized.vocab'
#         w2salience_f='../models/lda/w2entropy'


####context2vec-skipgram
        model_param_file='../models/context2vec/model_dir/MODEL-wiki.params.14?../models/wiki_all.model/wiki_all.sent.split.model'
#         model_param_file='../models/context2vec/model_dir/context2vec.ukwac.model.params?../models/wiki_all.model/wiki_all.sent.split.model'
        model_type='context2vec-skipgram'
#         context_rm_stopw=0
        n_result=20
    
    else:
        if len(sys.argv) < 5:
            print >> sys.stderr, "Usage: {0} <model_param_file> <model_type> <weight:{1}> <eval_data> <w2salience>"  .format (sys.argv[0],WEIGHT_DICT.items())
            sys.exit(1)
        
        model_param_file = sys.argv[1]
        model_type=sys.argv[2]
        
        if '-' in sys.argv[3]:
            weight,n_result=sys.argv[3].split('-')
            weight=WEIGHT_DICT[int(weight)]
            n_result=int(n_result)
        else:
            weight=WEIGHT_DICT[int(sys.argv[3])]
            n_result=20 #default is 20 top
            
#         context_rm_stopw=int(sys.argv[4])
        data =sys.argv[4]
        
        if len(sys.argv)>5:
            w2salience_f=argv[5]
        else:
            w2salience_f=None
    
    #gpu setup 
    gpu = -1 # todo: make this work with gpu

    if gpu >= 0:
        cuda.check_cuda_available()
        cuda.get_device(gpu).use()    
    xp = cuda.cupy if gpu >= 0 else np
    
    # logging
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    
    #choose model type
    print ('read model....')
    if model_type=='context2vec':
        #read in model
        
        model_reader = ModelReader(model_param_file)
        w = model_reader.w
        index2word = model_reader.index2word
        word2index=model_reader.word2index
        model = model_reader.model
        w_target=None
        word2index_target=None
        index2word_target=None
        context_rm_stopw=0
    elif model_type=='skipgram':
        model = gensim.models.Word2Vec.load(model_param_file)
        w=deepcopy(model.wv.vectors)
        #vector normalize for probe w embedding
        s = np.sqrt((w * w).sum(1))
        s[s==0.] = 1.
        w /= s.reshape((s.shape[0], 1))
        
        index2word=model.wv.index2word
        word2index={key: model.wv.vocab[key].index for key in model.wv.vocab}
        w_target=None
        word2index_target=None
        index2word_target=None
        context_rm_stopw=1
        
    elif model_type=='context2vec-skipgram':
        model_param_context,model_param_w2v=model_param_file.split('?')
        model_reader = ModelReader(model_param_context)
        w = model_reader.w
        index2word = model_reader.index2word
        word2index=model_reader.word2index
        model = model_reader.model
        
        model_w2v = gensim.models.Word2Vec.load(model_param_w2v)
        w_target=model_w2v.wv.vectors
        index2word_target=model_w2v.wv.index2word
        word2index_target={key: model_w2v.wv.vocab[key].index for key in model_w2v.wv.vocab}
        context_rm_stopw=0
    
    w2salience=None
    
    #remove stop words in target word space
    print ('filter words for target....')
    w,word2index,index2word=filter_w(w,word2index,index2word)
    if  index2word_target!=None:
        w_target,word2index_target,index2word_target=filter_w(w_target,word2index_target,index2word_target)
    
    #weight

#     if weight==TOP_MUTUAL_SIM of weight==TOP_CLUSTER_DENSITY:
    if weight==LDA:
        print ('load vectors and entropy')
        w2salience=pickle.load(open(w2salience_f))
    elif weight==INVERSE_W_FREQ:
        print ('load w2freq')
        w2salience=load_w2salience(w2salience_f,weight)
    elif weight==INVERSE_S_FREQ:
        print ('load w2freq')
        w2salience=load_w2salience(w2salience_f,weight)


    # remove context stop words
    if int(context_rm_stopw)==1:
        print ('filter words for context....')

        model=rm_stopw_context(model)
        
    
    print (model_param_file,model_type,weight,context_rm_stopw,data,w2salience_f)


read model....
Reading config file: ../models/context2vec/model_dir/MODEL-wiki.params.14
Config:  {'config_path': '../models/context2vec/model_dir/', 'model_file': 'MODEL-wiki.14', 'deep': 'yes', 'drop_ratio': '0.0', 'words_file': 'WORDS-wiki.targets.14', 'unit': '400'}
filter words for target....


In [26]:
#read in data
# data='./eval_data/data-chimeras/dataset.l6.fixed.test.txt.punct'
if data.split('/')[-2]== 'data-chimeras':
#         weight=WEIGHT_DICT[5]
#         print (weight)
        eval_chimera(data,model,model_type,n_result,w,index2word,weight,w2salience,w_target,word2index_target,index2word_target)
        

top_cluster_density
./eval_data/data-chimeras
4
canned sardines and  ___  between two slices of wholemeal bread and thinly spread flora original .
producing top 20 simwords
meatloaf: 0.5400038
sandwiches: 0.5396633
tamales: 0.538706
chicken: 0.53835166
spreads: 0.53812134
popcorn: 0.5368529
sliced: 0.53673893
mash: 0.53662497
squeezed: 0.5363477
sorbet: 0.53589183
sandwiched: 0.53515226
cooked: 0.5348494
stuffed: 0.5340092
couscous: 0.53323156
fritters: 0.5331776
sauerkraut: 0.5330202
mixtures: 0.532765
alternates: 0.5324085
pickles: 0.5319029
smothered: 0.5318118
top_cluster_density
0.1994159702048819
weight is 0.199415970205
erm ,  ___  , low fat dairy products , incidents of heart disease for those who have an olive oil rich diet .
producing top 20 simwords
dairies: 0.54627275
pharmaceuticals: 0.53947484
pharma: 0.53916913
probiotics: 0.5363117
medications: 0.5351843
freezers: 0.5344019
stocks: 0.53410417
toothpastes: 0.53363115
cigarettes: 0.53224295
antibiotics: 0.5318707
fertiliz

dryers: 2.410021397384447
heaters: 2.287674014874611
refrigerators: 2.2283185651140682
conditioners: 2.2060273912187327
dishwashers: 2.187360237146278
countertops: 2.177482950370545
washers: 2.1536678703113035
faucets: 2.143473133835979
stoves: 2.129653007026908
coolers: 2.1215162733073725
hvac: 2.0962491647500583
humidifiers: 2.0947642153382473
bathtubs: 2.08657882975143
cookers: 2.0821765788009454
freezers: 2.0783120229254655
exchangers: 2.076196342531757
lotions: 2.0741902278133315
refrigeration: 2.070755790352465
desiccant: 2.0455872291707946
heater: 2.0366521819376397
('probes', ['stove', 'microwave', 'kettle', 'cage', 'wastebin', 'leopard'])
('gold', ['2', '2.86', '1.71', '1.29', '1.29', '1.43'])
('model_predict', [0.6540423440345664, 0.4766230471553823, 0.39453497243949587, 0.28707659862069773, 0.26131489519003703, 0.17873685779156184])
spearman correlation is 0.753702346348
what a gorgeous man to work for , anna confided , as she and merrill stacked the  ___  later .
producing 

communication: 0.53766054
biomolecular: 0.5359215
telecommunication: 0.53123266
vulnerable: 0.5306935
hydronic: 0.52950984
signalling: 0.52887285
musculoskeletal: 0.52825975
redundant: 0.5273194
excretory: 0.52708215
unmanned: 0.5270074
sewage: 0.5266811
avionics: 0.52606964
telematic: 0.52559805
electronic: 0.52552056
microelectronic: 0.5251757
microelectromechanical: 0.52517474
electromechanical: 0.52451944
electrical: 0.5245029
fastening: 0.5240151
water: 0.52390796
top_cluster_density
0.16080082413759475
weight is 0.160800824138
and in  ___  went off in towns and cities nationwide almost simultaneously , leaving two people dead .
producing top 20 simwords
sloviansk: 0.5494989
pristina: 0.53434443
euromaidan: 0.5333552
sanatoriums: 0.53307927
desperation: 0.5323595
kramatorsk: 0.5313507
gjakova: 0.52725536
flames: 0.5268635
luhansk: 0.5267165
retaliation: 0.5264642
rioting: 0.52608454
retribution: 0.5260176
protest: 0.5254646
vain: 0.52530324
donbass: 0.5252157
riots: 0.524915
feriz

editorgiven1: 2.4424752150890514
editorsurname4: 2.404290819870354
laysummary: 2.3492642787893785
laydate: 2.28362183116889
publicationdate: 2.2274024553277636
editorsurname3: 2.2082984889022312
editorsurname1: 2.1871701043986844
editorsurname2: 2.18013616700515
bassoons: 2.0880302271351687
pyrams: 2.009511341965295
52-69: 1.9847324079739548
oboes: 1.982482501475976
muricidae: 1.9819544189069038
passeriformesfamily: 1.9807943009900322
florets: 1.962339309640857
pelecaniformesfamily: 1.9204965524550717
hindwings: 1.9036331262381307
authorsep: 1.9000158019482913
ugauga: 1.8902426761592022
officership: 1.8866460823467839
('probes', ['bagpipe', 'harmonica', 'whistle', 'shotgun', 'bear', 'bouquet'])
('gold', ['2.29', '2', '1.86', '1.57', '2.14', '1.86'])
('model_predict', [0.37552022762078674, 0.48450547774096503, 0.41249973568604076, 0.44769258268405965, 0.22866329515823347, 0.3077054648694734])
spearman correlation is -0.376851173174
the new corps of mcneillstown scored their first succes

producing top 20 simwords
men: 0.5306778
maidens: 0.5286087
buddhas: 0.52651554
rathas: 0.5248934
apostles: 0.5248148
sisters: 0.5244507
thieves: 0.52372116
englishmen: 0.52333766
lads: 0.52293724
lovers: 0.522708
families: 0.5226182
graces: 0.5220536
dwarves: 0.52196985
brothers: 0.5216378
gallants: 0.5212962
sages: 0.5210994
trees: 0.5205441
survivors: 0.5202488
elders: 0.51997113
strangers: 0.51983863
top_cluster_density
0.1447151748082831
weight is 0.144715174808
normalized weight: 
  [[0.25657269]
 [0.29555368]
 [0.26324719]
 [0.18462644]]
producing top 20 words for new embedding
(259235, 400)
producing top 20 simwords
passeriformesfamily: 2.3761747620931724
egrets: 2.3590897226275516
opossums: 2.3447148716473434
avocets: 2.3303920002670604
ibises: 2.315282252119032
quails: 2.260791403015749
pelecaniformesfamily: 2.2592394044058417
cormorants: 2.2567748903919447
woodpeckers: 2.255940439850131
spoonbills: 2.253546383564284
partridges: 2.2531385554554415
porcupines: 2.24027202535102

...

producing top 20 simwords
instrument: 0.5304002
organ: 0.5298997
instrumentation: 0.527622
pedal: 0.5263692
instruments: 0.5262388
training: 0.5244481
functioning: 0.5234674
music: 0.52337235
work: 0.52111053
flute: 0.52031195
anatomy: 0.52008253
operation: 0.5187718
technique: 0.5180018
workshop: 0.5179768
tuning: 0.5170567
equipment: 0.5167454
rudiments: 0.5160417
timpani: 0.5157255
choir: 0.5154243
transposition: 0.5153846
top_cluster_density
0.14045128233231427
weight is 0.140451282332
here  ___  are placed above the oboes and would thus give a clear brilliant " tang " to the chord .
producing top 20 simwords
trumpets: 0.5333894
strings: 0.5248232
chords: 0.52438587
notes: 0.5241466
volutes: 0.5232438
sixths: 0.522915
trills: 0.522349
horns: 0.52225304
cymbals: 0.519197
cadenzas: 0.5179724
arabesques: 0.517601
modulations: 0.51726645
variations: 0.516425
tempers: 0.5164192
bows: 0.51637644
triads: 0.5153483
couplets: 0.51462954
recorders: 0.51400024
dampers: 0.5136838
cadences: 0.5

oboes: 2.9150509094179613
bassoons: 2.8604519231752903
clarinets: 2.6258993969009286
timpani: 2.6028902228582886
continuo: 2.5783664960162147
arpeggios: 2.559257619700105
trombones: 2.54797954735505
violas: 2.527228693496569
contrabassoon: 2.5037110833844816
cellos: 2.501483355874346
12-string: 2.4278634421261445
glockenspiel: 2.4137916019642756
violins: 2.399888194164981
woodwinds: 2.393876723019968
octaves: 2.3915451918128667
trills: 2.3816676239152166
arpeggiated: 2.377483284640152
bassoon: 2.375167926643345
pizzicato: 2.3623870831769977
vibraphone: 2.3548593911035587
('probes', ['banjo', 'gorilla', 'whistle', 'worm', 'dress', 'pine'])
('gold', ['3.14', '2.57', '2.43', '2.14', '2', '2'])
('model_predict', [0.6133986675617449, 0.24877943704372651, 0.4395322087535497, 0.25822179548296176, 0.27426255159090535, 0.20855427133703996])
spearman correlation is 0.521793932087
the highly effective  ___  is satin finished lever - arm with a geared action and soft touch panels inset into the ar

cocoon: 0.5462168
container: 0.5424089
husk: 0.54221326
pot: 0.5367699
dough: 0.53444505
compost: 0.5331894
tart: 0.53148574
drupe: 0.52958775
batter: 0.5286207
receptacle: 0.52824605
leaf: 0.52677256
salad: 0.5266242
skillet: 0.52563107
casserole: 0.52527815
pulp: 0.5251222
omelette: 0.5250881
bun: 0.52495676
puree: 0.5247391
crumb: 0.52408457
oven: 0.523987
top_cluster_density
0.18343948474637273
weight is 0.183439484746
normalized weight: 
  [[0.23022655]
 [0.29898799]
 [0.23318821]
 [0.23759725]]
producing top 20 words for new embedding
(259235, 400)
producing top 20 simwords
cilantro: 2.4159868851961144
marinated: 2.4142038671665493
flavorings: 2.4134882181852104
coriander: 2.389239951117246
shallots: 2.3831453849178166
flavouring: 2.3498843107801686
flavoring: 2.3421656946066385
toppings: 2.3402707321738836
sauces: 2.322282070914658
syrups: 2.3097850340686135
caramelized: 2.3018194779779195
salads: 2.296249702290449
stewed: 2.2931772430752972
flavourings: 2.288771607779458
goreng

flouring: 0.5669532
fulling: 0.56203866
grist: 0.5525673
flour: 0.54787165
forge: 0.5446891
woolen: 0.54272336
flax: 0.5423984
puddling: 0.54228467
woollen: 0.54156166
cooperage: 0.5376648
carding: 0.5370254
fireclay: 0.5332699
ironstone: 0.5331704
grinding: 0.5330743
corn: 0.53304815
loom: 0.5326759
timber: 0.5323607
flint: 0.5321514
spinning: 0.53181225
brickmaking: 0.53095627
top_cluster_density
0.23537654352091125
weight is 0.235376543521
carbendazim , a hormone disrupting pesticide , was found above legal limits [ 2 ] in apricots , green beans  ___  .
producing top 20 simwords
3: 0.54926217
1-5: 0.544183
3-4: 0.5434875
2: 0.5420016
6: 0.54010135
5: 0.5383905
etc: 0.53798604
4: 0.5369412
03: 0.53652936
12-17: 0.5345086
22-24: 0.5344567
1-2: 0.534272
2-3: 0.53266644
7: 0.5326009
respectively: 0.53239584
5-7: 0.53222877
1-3: 0.531649
6-7: 0.5313092
ripened: 0.52987885
12-20: 0.52919996
top_cluster_density
0.23219465495491365
weight is 0.232194654955
what happened to the brown bowls w

producing top 20 simwords
kid: 0.53518254
book: 0.5323258
rug: 0.5293166
gem: 0.529003
patch: 0.52823204
vibe: 0.52822435
shawl: 0.528121
girl: 0.5279498
thing: 0.52750593
blanket: 0.52710855
bump: 0.527075
vid: 0.5267277
card: 0.52612984
twist: 0.52590054
costume: 0.52576846
dojo: 0.52554816
monster: 0.52540123
groove: 0.5247941
pen: 0.52366185
carpet: 0.52347374
top_cluster_density
0.12343127824221974
weight is 0.123431278242
normalized weight: 
  [[0.21166014]
 [0.33025929]
 [0.28704249]
 [0.17103808]]
producing top 20 words for new embedding
(259235, 400)
producing top 20 simwords
opossums: 2.25928755581345
egrets: 2.2528001321428124
avocets: 2.2445274065240923
insectivores: 2.235643437908537
skinks: 2.231625944945526
pelecaniformesfamily: 2.223605006065518
lizards: 2.2196028242146437
passeriformesfamily: 2.2158608961565833
tapirs: 2.200952121058342
gymnures: 2.1931448404381113
geckos: 2.1920909174358503
ibises: 2.1859947746696973
porcupines: 2.1830183496266518
mongooses: 2.1821134